In [1]:
# AirPilot-tuned PID controller, where PID gains remains constant 
import gym
from gym import spaces
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
from gazebo_msgs.msg import ModelStates, ModelState
from gazebo_msgs.srv import SetModelState
import rospy
import sys
from clover.srv import SetVelocity, SetVelocityRequest, SetRates, SetRatesRequest, Navigate, NavigateRequest
import numpy as np
from geometry_msgs.msg import Wrench, Vector3, PoseStamped
from mavros_msgs.srv import CommandBool, CommandTOL, SetMode
from std_msgs.msg import String
from mavros_msgs.msg import State
from std_srvs.srv import Empty
import tf.transformations as transformations
import math
import random 



def get_current_position():
    try:
        model_states = rospy.wait_for_message('/gazebo/model_states', ModelStates, timeout=10)
        PI_2 = math.pi / 6 #modified to prevent large angle
        #position_message = rospy.wait_for_message('/mavros/local_position/pose', PoseStamped, timeout=5)
        drone_index = model_states.name.index('clover')  
        position = model_states.pose[drone_index].position
        #print("Current position:", position.x, position.y, position.z)
        
        #check if the drone is flipped upside down
        orientation = model_states.pose[drone_index].orientation
        
        quaternion = [orientation.x, orientation.y, orientation.z, orientation.w]
        #angle = self.quaternion_to_inclined_angle(quaternion)

        roll, pitch, yaw = transformations.euler_from_quaternion(quaternion)
        flipped = abs(roll) > PI_2 or abs(pitch) > PI_2

        return  np.array([position.x, position.y, position.z])#, angle
    except rospy.ROSException as e:
        print("Failed to get position message within timeout:", str(e))
        return np.array([0, 0, 0]), True#, 0
    except rospy.ROSInterruptException:
        print("Node was shutdown")

def set_velocity(vx, vy, vz):
    velocity_request = SetVelocityRequest()
    velocity_request.vx = vx
    velocity_request.vy = vy
    velocity_request.vz = vz
    velocity_request.frame_id = 'map'
    velocity_request.auto_arm = 1
    try:
        response = set_velocity_service(velocity_request)
        #print(response, vx, vy, vz)
        
    except rospy.ServiceException as e:
        print("Service call failed: %s" % e)

rospy.init_node('jupyter_pid_controller', anonymous=True)
target_position = np.array([5, 5, 1.5])
current_position = get_current_position()
previous_error = np.array([0.0, 0.0, 0.0])
sum_error = np.array([0.0, 0.0, 0.0])
position_error = target_position - current_position
distance = np.linalg.norm(position_error)
Kpid = np.array([0.359, 0.0529, 0.0001])
set_velocity_service = rospy.ServiceProxy('/set_velocity', SetVelocity)
current_step = 0
step_to_success = 0

for _ in range(10000):  
    velocity = position_error * Kpid[0] + (position_error-previous_error)* Kpid[1] + (position_error + sum_error) * Kpid[2]
    normalized_v = np.array([x / (abs(x) + 1) for x in velocity]) # velocity ~ [-1,1]
    
    #print(Kpid)
    
    set_velocity(normalized_v[0], normalized_v[1],normalized_v[2])
    rospy.sleep(0.04)# need to be mverify: control frequency 250Hz?
    
    current_position = get_current_position()
    #print(current_position)
    previous_error = position_error
    sum_error += position_error
    position_error = target_position - current_position # update position error after taking a step
    current_step+=1
    if np.linalg.norm(position_error) <= 0.25: #self.success_threshold:
        step_to_success += 1
        
        if step_to_success == 50:
            average_speed  = distance / ((current_step-50)*0.04) 
            print("Reach the goal! Average flying speed:", average_speed, "m/s")
    else:
        step_to_success = 0
                
print("game is over")

Reach the goal! Average flying speed: 0.9347285690311818 m/s


ROSInterruptException: ROS shutdown request